## Proceso 4: Clasificación de sentimientos múltiples, como: alegría, ira, culpa, etc.

In [1]:
import re 
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

import sys
sys.path.append(sys.path[0] + '/..')

In [2]:
def read_data(file):
    data = []
    with open(file, 'r')as f:
        for line in f:
            line = line.strip()
            label = ' '.join(line[1:line.find("]")].strip().split())
            text = line[line.find("]")+1:].strip()
            data.append([label, text])
    return data

In [3]:
file = 'sentimientoMultiple_es.txt'
data = read_data(file)
print("Number of instances: {}".format(len(data)))

Number of instances: 7480


### tokenización y la generación de las características de una oración 

In [4]:
def ngram(token, n): 
    output = []
    for i in range(n-1, len(token)): 
        ngram = ' '.join(token[i-n+1:i+1])
        output.append(ngram) 
    return output

def create_feature(text, nrange=(1, 1)):
    text_features = [] 
    text = text.lower() 
    text_alphanum = re.sub('[^a-z0-9#]', ' ', text)
    for n in range(nrange[0], nrange[1]+1): 
        text_features += ngram(text_alphanum.split(), n)    
    text_punc = re.sub('[a-z0-9]', ' ', text)
    text_features += ngram(text_punc.split(), 1)
    return Counter(text_features)

### Función para almacenar las etiquetas, que se basarán en emociones como Alegría, Miedo, Ira, etc.:

In [5]:
def convert_label(item, name): 
    #print('item: ', item)
    #print('name: ', name)
    items = list(map(float, item.split()))
    label = ""
    for idx in range(len(items)): 
        if items[idx] == 1: 
            label += name[idx] + " "
    
    return label.strip()

emotions = ["alegría", 'miedo', "ira", "tristeza", "disgusto", "vergüenza", "culpa"]

X_all = []
y_all = []
for label, text in data:
    y_all.append(convert_label(label, emotions))
    X_all.append(create_feature(text, nrange=(1, 4)))

### Separación de datos en conjuntos de entrenamiento y prueba

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, random_state = 123)

def train_test(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    train_acc = accuracy_score(y_train, clf.predict(X_train))
    test_acc = accuracy_score(y_test, clf.predict(X_test))
    return train_acc, test_acc

from sklearn.feature_extraction import DictVectorizer
vectorizer = DictVectorizer(sparse = True)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

### Entrenamiento con cuatro modelos de aprendizaje automático. Se imprime resultados para la selección del mejor modelo 

In [7]:
svc = SVC()
lsvc = LinearSVC(random_state=123)
rforest = RandomForestClassifier(random_state=123)
dtree = DecisionTreeClassifier()

clifs = [svc, lsvc, rforest, dtree]

# train and test them 
print("| {:25} | {} | {} |".format("Classifier", "Training Accuracy", "Test Accuracy"))
print("| {} | {} | {} |".format("-"*25, "-"*17, "-"*13))
for clf in clifs: 
    clf_name = clf.__class__.__name__
    train_acc, test_acc = train_test(clf, X_train, X_test, y_train, y_test)
    print("| {:25} | {:17.7f} | {:13.7f} |".format(clf_name, train_acc, test_acc))

#clf_name = svc.__class__.__name__
#train_acc, test_acc = train_test(svc, X_train, X_test, y_train, y_test)
#print("| {:25} | {:17.7f} | {:13.7f} |".format(clf_name, train_acc, test_acc))



| Classifier                | Training Accuracy | Test Accuracy |
| ------------------------- | ----------------- | ------------- |
| SVC                       |         0.9097594 |     0.4699198 |


c:\tools\ides\anaconda\envs\entorno_prueba\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


| LinearSVC                 |         0.9988302 |     0.5514706 |
| RandomForestClassifier    |         0.9988302 |     0.5421123 |
| DecisionTreeClassifier    |         0.9988302 |     0.4251337 |


## Definición de la etiqueta emocional y emoji

In [8]:
l = ["alegría", 'miedo', "ira", "tristeza", "disgusto", "vergüenza", "culpa"]
l.sort()
label_freq = {}
for label, _ in data: 
    label_freq[label] = label_freq.get(label, 0) + 1

# print the labels and their counts in sorted order 
for l in sorted(label_freq, key=label_freq.get, reverse=True):
    print("{:10}({})  {}".format(convert_label(l, emotions), l, label_freq[l]))

emoji_dict = {"alegría":"😂", "miedo":"😱", "ira":"😠", "tristeza":"😢", "disgusto":"😒", "vergüenza":"😳", "culpa":"😳"}

alegría   (1. 0. 0. 0. 0. 0. 0.)  1084
ira       (0. 0. 1. 0. 0. 0. 0.)  1080
tristeza  (0. 0. 0. 1. 0. 0. 0.)  1079
miedo     (0. 1. 0. 0. 0. 0. 0.)  1078
disgusto  (0. 0. 0. 0. 1. 0. 0.)  1057
culpa     (0. 0. 0. 0. 0. 0. 1.)  1057
vergüenza (0. 0. 0. 0. 0. 1. 0.)  1045


## Obtiene letra de canción preprocesada

In [17]:
from bdd import mongoDB_cliente

# Maluma                HAWÁI
# EL BOBE / ALPEDUE     ELLA
canciones = mongoDB_cliente.obtenerColeccion('CANCIONES', {'cantante': 'EL BOBE / ALPEDUE', 'cancion': 'ELLA'})   
letra_preprocesada = canciones[0]['letra_preprocesada']
letra_cancion_pre = ''
for letra in letra_preprocesada:
    letra_cancion_pre = letra_cancion_pre + letra + " "
print(letra_cancion_pre)

yeah eh eh eh yeah yeah yeah yeah eh eh sufrí demasía prometí queda ltima vez equivoca ltimo primer novio abusador descubrió jangueo verdadero amor entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock auh austin baby lama auh glock glock pegue bolero quiera llega revuelca avispero par amiga fuego hablen amor hombre sincero piensa lejito reggie miller bien compra pues conoce dealer yeh ih prende pasar noche dice nadie paga bille arte magia magic cree amor labia tantas desilusiones sabía right corona contagia puso louis trajecito cartera chanel enamoró hacer quiera bien da abajo reggaetón retumba forever soltera tumba wuh marizon auh entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock glock laramercy gang fa rru entendió dio amor trident blep quiere chocolate rosa san valentín guasón maltrataba harley quinn ahora an

## Ejecución del modelo entrenado: SVC

In [18]:
#clifs = [svc, lsvc, rforest, dtree]
features = create_feature(letra_cancion_pre, nrange=(1, 4))
features = vectorizer.transform(features)
prediction = svc.predict(features)[0]
print( letra_cancion_pre,emoji_dict[prediction], prediction)

yeah eh eh eh yeah yeah yeah yeah eh eh sufrí demasía prometí queda ltima vez equivoca ltimo primer novio abusador descubrió jangueo verdadero amor entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock auh austin baby lama auh glock glock pegue bolero quiera llega revuelca avispero par amiga fuego hablen amor hombre sincero piensa lejito reggie miller bien compra pues conoce dealer yeh ih prende pasar noche dice nadie paga bille arte magia magic cree amor labia tantas desilusiones sabía right corona contagia puso louis trajecito cartera chanel enamoró hacer quiera bien da abajo reggaetón retumba forever soltera tumba wuh marizon auh entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock glock laramercy gang fa rru entendió dio amor trident blep quiere chocolate rosa san valentín guasón maltrataba harley quinn ahora an

## Ejecución del modelo entrenado: LSVC

In [19]:
#clifs = [svc, lsvc, rforest, dtree]
features = create_feature(letra_cancion_pre, nrange=(1, 4))
features = vectorizer.transform(features)
prediction = lsvc.predict(features)[0]
print( letra_cancion_pre,emoji_dict[prediction], prediction)

yeah eh eh eh yeah yeah yeah yeah eh eh sufrí demasía prometí queda ltima vez equivoca ltimo primer novio abusador descubrió jangueo verdadero amor entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock auh austin baby lama auh glock glock pegue bolero quiera llega revuelca avispero par amiga fuego hablen amor hombre sincero piensa lejito reggie miller bien compra pues conoce dealer yeh ih prende pasar noche dice nadie paga bille arte magia magic cree amor labia tantas desilusiones sabía right corona contagia puso louis trajecito cartera chanel enamoró hacer quiera bien da abajo reggaetón retumba forever soltera tumba wuh marizon auh entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock glock laramercy gang fa rru entendió dio amor trident blep quiere chocolate rosa san valentín guasón maltrataba harley quinn ahora an

## Ejecución del modelo entrenado: rforest

In [20]:
#clifs = [svc, lsvc, rforest, dtree]
features = create_feature(letra_cancion_pre, nrange=(1, 4))
features = vectorizer.transform(features)
prediction = rforest.predict(features)[0]
print( letra_cancion_pre,emoji_dict[prediction], prediction)

yeah eh eh eh yeah yeah yeah yeah eh eh sufrí demasía prometí queda ltima vez equivoca ltimo primer novio abusador descubrió jangueo verdadero amor entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock auh austin baby lama auh glock glock pegue bolero quiera llega revuelca avispero par amiga fuego hablen amor hombre sincero piensa lejito reggie miller bien compra pues conoce dealer yeh ih prende pasar noche dice nadie paga bille arte magia magic cree amor labia tantas desilusiones sabía right corona contagia puso louis trajecito cartera chanel enamoró hacer quiera bien da abajo reggaetón retumba forever soltera tumba wuh marizon auh entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock glock laramercy gang fa rru entendió dio amor trident blep quiere chocolate rosa san valentín guasón maltrataba harley quinn ahora an

# Ejecución del modelo entrenado: dtree

In [21]:
#clifs = [svc, lsvc, rforest, dtree]
features = create_feature(letra_cancion_pre, nrange=(1, 4))
features = vectorizer.transform(features)
prediction = dtree.predict(features)[0]
print( letra_cancion_pre,emoji_dict[prediction], prediction)

yeah eh eh eh yeah yeah yeah yeah eh eh sufrí demasía prometí queda ltima vez equivoca ltimo primer novio abusador descubrió jangueo verdadero amor entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock auh austin baby lama auh glock glock pegue bolero quiera llega revuelca avispero par amiga fuego hablen amor hombre sincero piensa lejito reggie miller bien compra pues conoce dealer yeh ih prende pasar noche dice nadie paga bille arte magia magic cree amor labia tantas desilusiones sabía right corona contagia puso louis trajecito cartera chanel enamoró hacer quiera bien da abajo reggaetón retumba forever soltera tumba wuh marizon auh entendió entendió amor cambió cambió noches jangueo balada dembow dembow entendió entendió bien amor cambió cambió nadie joda cartera glock glock laramercy gang fa rru entendió dio amor trident blep quiere chocolate rosa san valentín guasón maltrataba harley quinn ahora an